In [1]:
#importing Libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import seaborn as sns
from sklearn.metrics import r2_score
import pprint
from rdkit import Chem
from rdkit.Chem import Descriptors

#Reading the data & dropping uneeded columns

data = pd.read_csv("/Users/iainquinn/Crystalisation Internship/Data/DataRaw.csv")
data = data[[ "smiles", "Source", "Method", "Molecule", "solvent","induction_time" ,"Supersaturation"]].copy()

#Making two new columns

data['ln_t'] = np.log(data['induction_time'])
data['ln_(S-1)'] = np.log(data['Supersaturation']-1)

#Adding a new column called molecule-source

moleculesource_lst=[]

for index, row in data.iterrows():
    moleculesource_lst.append(row['Molecule'] + ' ' + row['Source'])
data['molecule_source'] = moleculesource_lst

#making two new datasets

data_anti = data.loc[data.Method == "Antisolvent"]
data_cool = data.loc[data.Method == "Cooling"]

#Showing the simplified data
#data.head()

#Making lists of each unique molecule in the datasheet

molecule_list = data.Molecule.unique()
molecule_list_anti = data_anti.Molecule.unique()
molecule_list_cool = data_cool.Molecule.unique()

#Making a list of each unique molecule-source object in the datasheet

molsource_list = np.unique(moleculesource_lst)
molsource_list_anti = data_anti.molecule_source.unique()
molsource_list_cool = data_cool.molecule_source.unique()

In [2]:
molecules_source=[]
score=[]
slope=[]
intercept=[]
median=[]
method=[]
smiles=[]

for entry in molsource_list_anti:
    data_set = data.loc[data.molecule_source == entry]
    Y = np.c_[data_set["ln_t"]]
    X = np.c_[data_set["ln_(S-1)"]]
    model = sklearn.linear_model.LinearRegression()
    model.fit(X, Y)
    prediction=model.predict(X)
    molecules_source.append(entry)
    score.append(r2_score(Y, prediction))
    slope.append(float(model.coef_))
    intercept.append(float(model.intercept_))
    median.append(data_set["Supersaturation"].median())
    method.append('antisolvent')
    smiles.append(str(data_set.iloc[0]['smiles']))
    
df_anti = pd.DataFrame(list(zip( molecules_source, slope, intercept, score, median, method, smiles)) , columns=['molecule_source','slope', 'intercept', 'r2','median', 'method', 'smiles'])

molecules_source=[]
score=[]
slope=[]
intercept=[]
median=[]
method=[]
smiles=[]

for entry in molsource_list_cool:
    data_set = data.loc[data.molecule_source == entry]
    Y = np.c_[data_set["ln_t"]]
    X = np.c_[data_set["ln_(S-1)"]]
    model = sklearn.linear_model.LinearRegression()
    model.fit(X, Y)
    prediction=model.predict(X)
    molecules_source.append(entry)
    score.append(r2_score(Y, prediction))
    slope.append(float(model.coef_))
    intercept.append(float(model.intercept_))
    median.append(data_set["Supersaturation"].median())
    method.append('cooling')
    smiles.append(str(data_set.iloc[0]['smiles']))
    
df_cool = pd.DataFrame(list(zip( molecules_source, slope, intercept, score, median, method, smiles)) , columns=['molecule_source','slope', 'intercept', 'r2','median', 'method', 'smiles'])

df = pd.concat([df_anti, df_cool], ignore_index=True)
#df.head()

In [7]:
#Removing all poorly correlated data

df = df.loc[df.r2 > 0.5]

good_list = df['molecule_source'].tolist()

df = df.reset_index(drop=True)

smiles_list=df.smiles.tolist()

df=df.drop(columns=['smiles'])

df.head()

,molecule_source,slope,intercept,r2,median,method
0,Abecarnil Beckmann_1999,-2.679235,3.495162,0.901418,1.773416,antisolvent
1,Cefuroxime Sodium Zhao_2012,-2.992770,3.383713,0.894575,1.457791,antisolvent
2,Glycine Ramakers 2020,-2.103595,4.009274,0.731073,1.370000,antisolvent
3,L-Histidine Liu_2018,-1.120890,5.368310,0.817177,3.112457,antisolvent
4,Paracetamol O'Ciardha_2011,-2.393867,4.126005,0.567710,1.245241,antisolvent


In [8]:
mol_descriptors = [desc[0] for desc in Descriptors.descList]
dict_list=[]

for smile in smiles_list:
    my_dict={}
    for desc in mol_descriptors:
        a = "b=Descriptors." + desc + "(Chem.MolFromSmiles(smile))"
        exec(a)
        my_dict[desc]=b
    my_dict['smiles']=smile
    dict_list.append(my_dict)
        
df_desc = pd.DataFrame(dict_list)
#df_desc.head(20)

In [10]:
new_df=df.join(df_desc)

In [13]:
#Making a correlation matrix for the slope

corr_matrix= new_df.corr()
corr=corr_matrix["slope"].sort_values(ascending=False)
corr[0:5]

slope          1.000000
intercept      0.623847
BCUT2D_MRHI    0.476903
fr_aniline     0.475998
SlogP_VSA10    0.426702
Name: slope, dtype: float64

In [14]:
#Making a correlation matrix for the median

corr_matrix= new_df.corr()
corr=corr_matrix["median"].sort_values(ascending=False)
corr[0:5]

median                 1.000000
MaxAbsPartialCharge    0.533690
PEOE_VSA8              0.530252
MaxPartialCharge       0.477536
fr_imidazole           0.447682
Name: median, dtype: float64